### API Data Report
Jaclyn Bell</br>
10-12-2025

### Hypothesis
Posts with hashtags get higher visibility (likes + reposts) than posts without hashtags.
### Theoretical Application
Hashtags increase the change for posts to be seen outside of a follower base. I search hashtags to narrow down my search, and often come across new content I wouldn't have seen otherwise.

### Statisitcal Application
I am hoping to be able to test this by:
- checking for the presence of hashtags
- looking at the engagement of likes and reposts
- I can analyze it by calculating average engagement for hashtag vs non-hashtag posts
- I do a t-test to show show the results visually in a bar chart.

### Bluesky API endpoints

I am going to do this by using public endpoints. I will try to get this data by using these endpoints:

1. app.bsky.feed.searchPosts- to let me find posts using hashtag terms. With fields like, likeCount, repostCount, and replyCount.
2. app.bsky.feed.getPosts- I will us the URIs to get more detail of posts engagement numbers.

### Why are these endpoints suitable to test my hypothesis?
- app.bsky.feed.searchPosts is suitable because it allows me to query by keyword/hashtags. This will show me engagement numbers for likes, reposts, etc. I can use this to build a dataset of posts that contain hashtags.
- app.bsky.feed.getPosts is suitable because it pulls data from the posts I have URIs for. This gives me engagement counts that are up to date.

### How this data is Reliable
This data can be reliable because its pulling the data from Bluesky servers. And using the query I can target hashtagged content directly.


### How this data is Unreliable
I can be unreliable because searchPosts doesn't return all posts, search endpoints usually favor the most recent posts, and it doesn't give you full coverage over a long period of time.  


### Caveats for Testing the Hypothesis
Some hashtags are used more than others, so this could significatly change my dataset based on what I query. The samples might be biased towards more popular content.

# Data Collection

In [224]:
import requests
import time
import json as js
import pandas as pd

In [225]:
# Step 1: Authenticate with Bluesky
login_url = "https://bsky.social/xrpc/com.atproto.server.createSession"
credentials = {
    "identifier": "jbell85.bsky.social",  
    "password": "ylr6-tyg2-mi32-ylp5"          
}
session = requests.post(login_url, json=credentials)

if session.status_code != 200:
    raise Exception("Login failed: " + session.text)

access_jwt = session.json()["accessJwt"]  # token for authenticated requests
headers = {"Authorization": f"Bearer {access_jwt}"}

# Endpoint 1: 
app.bsky.feed.searchPosts

In [226]:
#endpoint = f"{BASE_URL}/app.bsky.feed.searchPosts"
endpoint = f"{BASE_URL}/app.bsky.feed.searchPosts"
headers = {"User-Agent": "Data Report (+jbell58@kent.edu)"}
params = {
"q": "Haircolor",  # keyword without hashtag
    "limit": 10
}

resp = requests.get(endpoint, params=params, headers=headers, timeout=30)
non_hashtag_posts = response_no.json()["posts"]
print("Status:", resp.status_code)

data = resp.json()

print("Top-level keys:", list(data.keys()))

Status: 200
Top-level keys: ['posts', 'cursor']


In [227]:
posts = data.get("posts", [])
print(posts)

feeds = data.get("feeds", [])
#print(feeds)

[{'uri': 'at://did:plc:trwxsj4d7kxadbxyghqdi7wk/app.bsky.feed.post/3m2z35gimtz2g', 'cid': 'bafyreib2q4jmaju2a36zjjqxvbkyaovy5fpoosjibsrondij4q5tgp5hx4', 'author': {'did': 'did:plc:trwxsj4d7kxadbxyghqdi7wk', 'handle': 'spellboundwigs.bsky.social', 'displayName': '', 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:trwxsj4d7kxadbxyghqdi7wk/bafkreifm2nqbga74m4nsa27acsfn767tnucscexslaadfspkzhr6irlk7m@jpeg', 'associated': {'activitySubscription': {'allowSubscriptions': 'followers'}}, 'labels': [], 'createdAt': '2024-12-11T22:52:38.542Z'}, 'record': {'$type': 'app.bsky.feed.post', 'createdAt': '2025-10-12T16:02:23.560131Z', 'embed': {'$type': 'app.bsky.embed.images', 'images': [{'alt': '', 'aspectRatio': {'height': 754, 'width': 754}, 'image': {'$type': 'blob', 'ref': {'$link': 'bafkreiark63ax3j5b6wvgzh5rco4xfpigvkraicnxkucue2hallltepuvu'}, 'mimeType': 'image/png', 'size': 947443}}]}, 'facets': [{'features': [{'$type': 'app.bsky.richtext.facet#tag', 'tag': 'believeinyourownmagic'}], 

In [228]:
# Step 4: Extract key fields into DataFrame
def extract_posts(posts, has_hashtag):
    rows = []
    for p in posts:
        text = p.get("record", {}).get("text", "")
        like_count = p.get("likeCount", 0)
        repost_count = p.get("repostCount", 0)
        reply_count = p.get("replyCount", 0)
        rows.append({
            "text": text,
            "likes": like_count,
            "reposts": repost_count,
            "replies": reply_count,
            "engagement": like_count + repost_count,
            "has_hashtag": has_hashtag
        })
    return rows

data = extract_posts(hashtag_posts, True) + extract_posts(non_hashtag_posts, False)
df = pd.DataFrame(data)


In [229]:
# Step 5: Show first rows
print(df.head(20))

                             text  likes  reposts  replies  engagement  has_hashtag
0   Mid October 3 Day Sale Hap...      0        0        0           0         True
1   I got new hair~🫧🫧🩵🩵🤍🤍 Yay✌...      9        1        0          10         True
2   Split decision\n\nCredit t...      0        0        0           0         True
3      I have an idea\n#haircolor      3        0        0           3         True
4   This red hot melt ain’t fo...      0        0        0           0         True
5   Transform your hair with A...      0        0        0           0         True
6   Got a haircut yesterday. A...      1        0        0           1         True
7   All Trimmed Up, I Have A J...    130        2        7         132         True
8   I changed hair style✨\nCut...     11        0        1          11         True
9   New hair color!!!!\n\n#new...      9        1        4          10         True
10  After months of faded colo...      7        0        1           7      

In [230]:

posts = response.json()["posts"]         # flat list; each item is a post
rows = []
for p in posts:
    stats = {
        "uri": p.get("uri"),
        "cid": p.get("cid"),
        "createdAt": p.get("record", {}).get("createdAt"),
        "text": p.get("record", {}).get("text", ""),
        "author_handle": p.get("author", {}).get("handle"),
        "author_did": p.get("author", {}).get("did"),
        "author_displayName": p.get("author", {}).get("displayName"),
        "likes": p.get("likeCount", 0),
        "reposts": p.get("repostCount", 0),
        "replies": p.get("replyCount", 0),
        "engagement": p.get("likeCount", 0) + p.get("repostCount", 0),
        "has_hashtag": ("True")
    }
    rows.append(stats)

df = pd.DataFrame(rows)

print(df[["uri","author_did","reposts","likes","has_hashtag"]].head(50))


                              uri                     author_did  reposts  likes has_hashtag
0   at://did:plc:trwxsj4d7kxad...  did:plc:trwxsj4d7kxadbxygh...        0      0        True
1   at://did:plc:7adgp3kwzd6pu...  did:plc:7adgp3kwzd6purwcnk...        1      9        True
2   at://did:plc:rvnaompvhxok5...  did:plc:rvnaompvhxok53cgh6...        0      0        True
3   at://did:plc:b4n5r4ogos2d5...  did:plc:b4n5r4ogos2d5p5rpv...        0      3        True
4   at://did:plc:rvnaompvhxok5...  did:plc:rvnaompvhxok53cgh6...        0      0        True
5   at://did:plc:6poofqa3v2wdc...  did:plc:6poofqa3v2wdcpggkz...        0      0        True
6   at://did:plc:c72gv6na3b44l...  did:plc:c72gv6na3b44l6it3p...        0      1        True
7   at://did:plc:rishlllijgwzj...  did:plc:rishlllijgwzjw36c7...        2    130        True
8   at://did:plc:7adgp3kwzd6pu...  did:plc:7adgp3kwzd6purwcnk...        0     11        True
9   at://did:plc:gbeycxxf5vnup...  did:plc:gbeycxxf5vnupkstyg...      

# Endpoint 2:
app.bsky.feed.getPosts

In [231]:
## Let us get profile data for all the authors from the previous feed
# get unique author ids which is dids
unique_dids = posts_df["author_did"].dropna().unique().tolist()
#print(unique_dids)

# Get author profiles for these dids
all_profiles = []
for d in unique_dids:
    #print(js.dumps(d, indent=2))
    params = []
    params.append(("feed", d))
    #print(d)
    r = requests.get(f"{BASE_URL}/app.bsky.feed.getPosts", params=params, timeout=30)
    data = r.json()
    #print(js.dumps(data, indent=2))

    # Append this profile in our list 
    # flatten tha data for profile
    all_profiles.append({
        "author_handle": p.get("author", {}).get("handle"),
        "author_did": p.get("author", {}).get("did"),
        "author_displayName": p.get("author", {}).get("displayName"),
    })

all_profiles_df = pd.DataFrame(all_profiles)
# This will take a while to load !
all_profiles_df.head(5)

KeyError: 'author_did'